In [12]:
#data science and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

#creation of dataset
import _lib.ml_workflow.create_dataset as cds
from _lib.raman_lib.misc import load_data

#quality control
import _lib.ml_workflow.quality_control as qc
from _lib.raman_lib.preprocessing import RangeLimiter
from _lib.raman_lib.visualization import plot_spectra_peaks
from _lib.raman_lib.spectra_scoring import score_names

#preprocessing
from _lib.ml_workflow.preprocess_data import preprocess

#model creation
from sklearn.model_selection import LeaveOneGroupOut, cross_validate, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cluster import FeatureAgglomeration
from sklearn.decomposition import PCA, NMF
from sklearn.pipeline import Pipeline
from _lib.raman_lib.preprocessing import PeakPicker

#file handling
from pathlib import Path
import os

In [13]:
# define the paths to all experiment data
# each dir contains all the files associated with that experiment
# each file has a prefix that indicates the group and the measurement --> group_measurement_.....
path_etoposide = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide"
path_resveratrol = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Resveratrol"
path_both = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Both"
path_control = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Control"

# Define parameters
In order to function properly, the provided code depends on predefined parameters like output paths, limits and thresholds for the quality control, ...
## Define data paths
Define the location of the data, and where quality-controlled and preprocessed data should be stored. Both of them rely on a unique file-prefix that describes the data being analyzed.

In [14]:
FILE_PREFIX = "HL_428_E_R"
DATASET_OUT = "./data/" + FILE_PREFIX + ".csv"
RESULT_DIR = "./result/" + FILE_PREFIX
QC_OUT = RESULT_DIR + "/" + FILE_PREFIX + "_qc.csv"
PREP_OUT = RESULT_DIR + "/" + FILE_PREFIX + "_preprocessed.csv"


## Define quality scoring parameters
The quality control only uses peaks in a given interval, recognizes peaks via a filter (Sav-Gol) and scores them based on some metrics. Finally, the best N spectra are selected.
### Spectral Range Limits

In [15]:
QC_LIM_LOW = 450
QC_LIM_HIGH = 1650

### Peak Detection

In [16]:
QC_WINDOW = 35
QC_THRESHOLD = 0.001
QC_MIN_HEIGHT = 50

### Scoring

In [17]:
QC_SCORE = 1
QC_PEAKS = 1

### Number of spectra to keep

In [18]:
QC_NUM = 300

## Define Preprocessing Parameter
### Spectral Range Limits

In [19]:
PREP_LIM_LOW = QC_LIM_LOW
PREP_LIM_HIGH = QC_LIM_HIGH

### Window-width for smoothing

In [20]:
PREP_WINDOW = 15

## Settings for Cross Validation

In [21]:
SCORING = ['roc_auc', 'accuracy', 'f1']
N_TRIALS = 20
N_FOLDS = 5
N_CORES = -1

# Create the dataset
Create the dataset using the implementation provided by D. Zimmermann.
For the creation of the dataset, the two source dirs, as well as the desired labels are needed.
Furthermore, an output directory is needed, to store the created dataset 

In [22]:
datadir = Path(DATASET_OUT).parent
if not os.path.exists(datadir):
    os.makedirs(datadir)

dataset = cds.create_dataset([path_etoposide, path_resveratrol], ['Etoposide', 'Resveratrol'], grouped=True)
dataset.to_csv(DATASET_OUT, index=False)

root - INFO - Loading data
root - INFO - Loading data
root - INFO - Loading data
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Resveratrol
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Resveratrol
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Resveratrol
root - INFO - Finished loading data.
root - INFO - Finished loading data.
root - INFO - Finished loading data.


# Do quality control
Asses the spectra based on their quality, and remove low quality spectra

In [23]:
path_in = Path(DATASET_OUT)
path_out = Path(RESULT_DIR)

if not os.path.exists(path_out):
    os.makedirs(path_out)

path_out_data = path_out / (path_in.stem + "_qc.csv")
path_out_scores = path_out / (path_in.stem + "_qc_scores.csv")

data = pd.read_csv(path_in)

data_out, _, score_dict = qc.score_sort_spectra(data,
                                                n=QC_NUM,
                                                limits=[QC_LIM_LOW, QC_LIM_HIGH],
                                                bl_method="asls",
                                                sg_window=QC_WINDOW,
                                                threshold=QC_THRESHOLD,
                                                min_height=QC_MIN_HEIGHT,
                                                score_measure=QC_SCORE,
                                                n_peaks_influence=QC_PEAKS,
                                                detailed=True)

visualize = False
if visualize:
    data_vis = data.drop(columns=["label", "file", "group"]).values.astype(float)
    wns_vis = data.drop(columns=["label", "file", "group"]).columns.astype(float)

    rl = RangeLimiter(lim=[QC_LIM_LOW, QC_LIM_HIGH],
                      reference=wns_vis)

    data_rl = rl.fit_transform(data_vis)
    wns_rl = wns_vis[rl.lim_[0]:rl.lim_[1]]

    plot_spectra_peaks(wns_rl,
                       data_rl,
                       score_dict["peak_pos"],
                       labels=score_dict["total_scores"])

data_out.to_csv(path_out_data, index=False)

pd.DataFrame({score_names[QC_SCORE]: score_dict["intensity_scores"],
              "N Peaks": score_dict["peak_scores"]}).to_csv(
    path_out_scores, index=False
)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Preprocess the data

In [24]:
path_in = Path(QC_OUT)
path_out = Path(RESULT_DIR)

filename = path_in.stem.removesuffix("_qc")

if not os.path.exists(path_out):
    os.makedirs(path_out)

path_out = path_out / (filename + "_preprocessed.csv")

data = load_data(QC_OUT)

# save groups and remove column, otherwise preprocess won't work
groups = np.asarray(data.group)
data = data.drop(columns=["group"])

data_prep = preprocess(data, limits=[PREP_LIM_LOW, PREP_LIM_HIGH], sg_window=PREP_WINDOW)

#add groups again
data_prep.insert(2, "group", groups)

data_prep.to_csv(path_out, index=False)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Implement GroupKFold CV
## LDA Dimensionality Reduction 

In [25]:
path_in = PREP_OUT
#path_out = Path(args.out)

#filename = path_in.stem

data = load_data(path_in)

X = data.drop(columns=["label", "file", "group"])
wns = np.asarray(X.columns.astype(float))
X = np.asarray(X)
y = np.array(data.label)
y, y_key = pd.factorize(y)

#split the dataset according to the groups
groups = np.array(data.group)
logo = LeaveOneGroupOut()



### Baseline with LDA alone

In [26]:
clf = LinearDiscriminantAnalysis()
cross_validate(clf, X, y, cv=logo, groups=groups, scoring=SCORING)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


{'fit_time': array([0.18943906, 0.15541911, 0.17584205]),
 'score_time': array([0.00513721, 0.00258589, 0.00301075]),
 'test_roc_auc': array([0.73047786, 0.72267298, 0.6881299 ]),
 'test_accuracy': array([0.67487685, 0.71957672, 0.67307692]),
 'test_f1': array([0.7027027 , 0.70056497, 0.69642857])}

### Feature Selection with PCA followed by LDA

In [27]:
param_grid = {"pca__n_components": range(
    1, 51, 10
)}
clf = Pipeline([("pca", PCA()),
                ("lda", LinearDiscriminantAnalysis())])
#cross_validate(clf, X, y, cv=logo, groups=groups, scoring=SCORING)
grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy').fit(X, y, groups=groups)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### Non-Negative Matrix Factorization

In [28]:
"""clf = Pipeline([("nmf", NMF(init="nndsvda", tol=1e-2, max_iter=5000)),
                ("lda", LinearDiscriminantAnalysis())])
cross_validate(clf, X, y, cv=logo, groups=groups, scoring=SCORING)"""

'clf = Pipeline([("nmf", NMF(init="nndsvda", tol=1e-2, max_iter=5000)),\n                ("lda", LinearDiscriminantAnalysis())])\ncross_validate(clf, X, y, cv=logo, groups=groups, scoring=SCORING)'

### Feature Agglomeration

In [29]:
param_grid = {"agglo__n_clusters": range(
    5, 41, 5
)}
clf = Pipeline([("agglo", FeatureAgglomeration(connectivity=np.diag(np.ones(len(wns))) +
                                                            np.diag(np.ones(len(wns) - 1), 1) +
                                                            np.diag(np.ones(len(wns) - 1), -1))),
                ("lda", LinearDiscriminantAnalysis())])
#cross_validate(clf, X, y, cv=logo, groups=groups, scoring=SCORING)
grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy').fit(X, y, groups=groups)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:
 Pipeline(steps=[('agglo',
                 FeatureAgglomeration(connectivity=array([[1., 1., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 1., 1.]]),
                                      n_clusters=25)),
                ('lda', LinearDiscriminantAnalysis())])

 The best score across ALL searched params:
 0.7103142440786119

 The best parameters across ALL searched params:
 {'agglo__n_clusters': 25}


### PeakPicker

In [30]:
clf = Pipeline([("peaks", PeakPicker()),
                ("lda", LinearDiscriminantAnalysis())])
cross_validate(clf, X, y, cv=logo, groups=groups, scoring=SCORING)


{'fit_time': array([0.003613  , 0.00379205, 0.00358009]),
 'score_time': array([0.00192499, 0.00212193, 0.00187802]),
 'test_roc_auc': array([0.72902098, 0.64649538, 0.69578201]),
 'test_accuracy': array([0.6453202 , 0.64550265, 0.67788462]),
 'test_f1': array([0.5862069 , 0.62146893, 0.74524715])}

In [31]:
param_grid = {"peaks__min_dist": range(
    10, 151, 10
)}
clf = Pipeline([("peaks", PeakPicker()),
                ("lda", LinearDiscriminantAnalysis())])
grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy').fit(X, y, groups=groups)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)


 Results from Grid Search 

 The best estimator across ALL searched params:
 Pipeline(steps=[('peaks', PeakPicker(min_dist=40)),
                ('lda', LinearDiscriminantAnalysis())])

 The best score across ALL searched params:
 0.6695364288755092

 The best parameters across ALL searched params:
 {'peaks__min_dist': 40}


## Regularized Models
### Logistic Regression L1 Penalty

In [32]:
param_grid = {
    "logreg__C": np.logspace(-2, 1, 16)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(solver="liblinear",
                                  penalty="l1", max_iter=1000, random_state=41))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy').fit(X, y, groups=groups)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:
 Pipeline(steps=[('scaler', StandardScaler()),
                ('logreg',
                 LogisticRegression(C=1.584893192461114, max_iter=1000,
                                    penalty='l1', random_state=41,
                                    solver='liblinear'))])

 The best score across ALL searched params:
 0.7132471615230237

 The best parameters across ALL searched params:
 {'logreg__C': 1.584893192461114}


### Logistic Regression L2 Penalty

In [33]:
param_grid = {
    "logreg__C": np.logspace(-5, 1, 13)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(solver="liblinear",
                                  penalty="l2", max_iter=1000, random_state=51))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy').fit(X, y, groups=groups)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:
 Pipeline(steps=[('scaler', StandardScaler()),
                ('logreg',
                 LogisticRegression(C=0.001, max_iter=1000, random_state=51,
                                    solver='liblinear'))])

 The best score across ALL searched params:
 0.7140816293115143

 The best parameters across ALL searched params:
 {'logreg__C': 0.001}


### Linear SVM L1 Penalty

In [34]:
param_grid = {
    "svm__C": np.logspace(-3, 0, 16)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", LinearSVC(penalty="l1", dual=False, max_iter=10000))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy').fit(X, y, groups=groups)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:
 Pipeline(steps=[('scaler', StandardScaler()),
                ('svm',
                 LinearSVC(C=0.15848931924611143, dual=False, max_iter=10000,
                           penalty='l1'))])

 The best score across ALL searched params:
 0.7065146964572252

 The best parameters across ALL searched params:
 {'svm__C': 0.15848931924611143}


### Linear SVM L2 Penalty

In [35]:
param_grid = {
    "svm__C": np.logspace(-5, -1, 13)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", LinearSVC(penalty="l2", max_iter=5000))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy').fit(X, y, groups=groups)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:
 Pipeline(steps=[('scaler', StandardScaler()),
                ('svm', LinearSVC(C=0.00021544346900318823, max_iter=5000))])

 The best score across ALL searched params:
 0.7227783391576494

 The best parameters across ALL searched params:
 {'svm__C': 0.00021544346900318823}


## Tree-based models
### Basic Decision Tree


In [36]:
param_grid = {
    "ccp_alpha": np.logspace(-3, -1, 9)
}
clf = DecisionTreeClassifier(random_state=653)

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy').fit(X, y, groups=groups)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:
 DecisionTreeClassifier(ccp_alpha=0.03162277660168379, random_state=653)

 The best score across ALL searched params:
 0.637769052998938

 The best parameters across ALL searched params:
 {'ccp_alpha': 0.03162277660168379}


### Random Forest

In [37]:
param_grid = {
    "colsample_bytree": np.linspace(0.01, 0.2, 20)
}
clf = LGBMClassifier(boosting_type="rf",
                     subsample=0.8,
                     subsample_freq=1,
                     max_bin=10,
                     max_depth=8,
                     random_state=2434)

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy').fit(X, y, groups=groups)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:
 LGBMClassifier(boosting_type='rf', colsample_bytree=0.12, max_bin=10,
               max_depth=8, random_state=2434, subsample=0.8, subsample_freq=1)

 The best score across ALL searched params:
 0.7014201741213236

 The best parameters across ALL searched params:
 {'colsample_bytree': 0.12}


### Gradient-boosted Decision Tree

In [38]:
param_grid = {
    "learning_rate": np.linspace(0.01, 0.2, 20)
}
clf = LGBMClassifier(colsample_bytree=0.2,
                     max_bin=10,
                     max_depth=5,
                     random_state=6233)

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy').fit(X, y, groups=groups)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:
 LGBMClassifier(colsample_bytree=0.2, learning_rate=0.17, max_bin=10,
               max_depth=5, random_state=6233)

 The best score across ALL searched params:
 0.7163712989862415

 The best parameters across ALL searched params:
 {'learning_rate': 0.17}
